In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as od
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
sample_submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
data = pd.concat([train, test], sort=False)

In [ ]:
data = data[['SalePrice','OverallQual', 'YearBuilt', 'TotalBsmtSF', 'GrLivArea', 'GarageCars', 'FullBath']]
data.head()

In [ ]:
data['YearOld'] = 2020 - data['YearBuilt']
del data['YearBuilt']
data.head()

In [ ]:
print(data.isnull().sum())

In [ ]:
data['TotalBsmtSF'].fillna(data['TotalBsmtSF'].mean(), inplace=True)

In [ ]:
data['GarageCars'].fillna(data['GarageCars'].mean(), inplace=True)

In [ ]:
train = data[:len(train)]
test = data[len(train):]

In [ ]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [ ]:
train.head()

In [ ]:
sns.distplot(train['SalePrice'])

In [ ]:
sns.distplot(np.log(train['SalePrice']))

In [ ]:
train['SalePrice'] = np.log(train['SalePrice'])
train

In [ ]:
test.head()

In [ ]:
y_train = train['SalePrice']
X_train = train.drop('SalePrice', axis = 1)
X_test = test.drop('SalePrice', axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

num_round=5000
evallist = [(dvalid, 'eval'), (dtrain, 'train')]

evals_result = {}

param = {'max_depth':3, 'eta':0.01, 'objective':'reg:squarederror'}

bst = xgb.train(param, dtrain, num_round, evallist, evals_result=evals_result, early_stopping_rounds=1000)

In [ ]:
dtest = xgb.DMatrix(X_test)

In [ ]:
sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sub['SalePrice'] = np.exp(bst.predict(dtest))
sub.to_csv('submission.csv', index=False)